## Based recommandation system

In [ ]:
import os
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
import rake_nltk
from rake_nltk import Rake


In [381]:

os.chdir("C:/Users/gaeta/Desktop/netflix_recommandation_system")

In [382]:
movies_names = pd.read_csv("data/movies.csv")
movies_names.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [383]:
ratings_data= pd.read_csv("data/ratings.csv")
ratings_data.head(5)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [384]:
movie_data =  pd.merge(ratings_data , movies_names , on= 'movieId')
movie_data.head(5)

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,7,31,3.0,851868750,Dangerous Minds (1995),Drama
2,31,31,4.0,1273541953,Dangerous Minds (1995),Drama
3,32,31,4.0,834828440,Dangerous Minds (1995),Drama
4,36,31,3.0,847057202,Dangerous Minds (1995),Drama


## Critères de recommandation 
##### 1- film avec le plus d'avis
##### 2- nombre de vues

## Moyenne des avis de films

In [385]:
movie_data.groupby('title')['rating'].mean().head()

title
"Great Performances" Cats (1998)           1.750000
$9.99 (2008)                               3.833333
'Hellboy': The Seeds of Creation (2004)    2.000000
'Neath the Arizona Skies (1934)            0.500000
'Round Midnight (1986)                     2.250000
Name: rating, dtype: float64

In [386]:
movie_data.groupby('title')['rating'].mean().sort_values(ascending=False).head(10)

title
Ivan Vasilievich: Back to the Future (Ivan Vasilievich menyaet professiyu) (1973)    5.0
Alien Escape (1995)                                                                  5.0
Boiling Point (1993)                                                                 5.0
Bone Tomahawk (2015)                                                                 5.0
Borgman (2013)                                                                       5.0
Survive and Advance (2013)                                                           5.0
Supercop 2 (Project S) (Chao ji ji hua) (1993)                                       5.0
Last Dragon, The (1985)                                                              5.0
Boy Crazy (2009)                                                                     5.0
Brandon Teena Story, The (1998)                                                      5.0
Name: rating, dtype: float64

## comptage de film

In [387]:
movie_data.groupby('title')['rating'].count().sort_values(ascending=False).head(10)

title
Forrest Gump (1994)                          341
Pulp Fiction (1994)                          324
Shawshank Redemption, The (1994)             311
Silence of the Lambs, The (1991)             304
Star Wars: Episode IV - A New Hope (1977)    291
Jurassic Park (1993)                         274
Matrix, The (1999)                           259
Toy Story (1995)                             247
Schindler's List (1993)                      244
Terminator 2: Judgment Day (1991)            237
Name: rating, dtype: int64

### Ajout du comptage des films a notre table

In [388]:
movie_data[['title', 'rating']]

,title,rating
0,Dangerous Minds (1995),2.5
1,Dangerous Minds (1995),3.0
2,Dangerous Minds (1995),4.0
3,Dangerous Minds (1995),4.0
4,Dangerous Minds (1995),3.0
...,...,...
99999,War of the Worlds (2005),2.5
100000,"Box, The (2009)",3.5
100001,Pie in the Sky (1996),3.0
100002,Summer Catch (2001),1.0


#### calcul de la moyenne des avis

In [389]:
ratings_mean_count = pd.DataFrame(movie_data.groupby('title')['rating'].mean())
ratings_mean_count

,rating
title,
"""Great Performances"" Cats (1998)",1.750000
$9.99 (2008),3.833333
'Hellboy': The Seeds of Creation (2004),2.000000
'Neath the Arizona Skies (1934),0.500000
'Round Midnight (1986),2.250000
...,...
xXx (2002),2.478261
xXx: State of the Union (2005),1.000000
¡Three Amigos! (1986),3.258065


##### Calcul du nombre d'évaluations et la note moyenne ci-dessus

In [390]:
ratings_mean_count['rating_counts'] = pd.DataFrame(movie_data.groupby('title')['rating'].count())

In [391]:
ratings_mean_count.head(5)

,rating,rating_counts
title,,
"""Great Performances"" Cats (1998)",1.750000,2
$9.99 (2008),3.833333,3
'Hellboy': The Seeds of Creation (2004),2.000000,1
'Neath the Arizona Skies (1934),0.500000,1
'Round Midnight (1986),2.250000,2


In [392]:
ratings_mean_count['rating'] = round(ratings_mean_count['rating'], 1) #arondir a un chiffre apres la virgule

In [393]:
ratings_mean_count.head(5)

,rating,rating_counts
title,,
"""Great Performances"" Cats (1998)",1.8,2
$9.99 (2008),3.8,3
'Hellboy': The Seeds of Creation (2004),2.0,1
'Neath the Arizona Skies (1934),0.5,1
'Round Midnight (1986),2.2,2


In [394]:
ratings_mean_count =  ratings_mean_count[(ratings_mean_count['rating']> 3 ) & (ratings_mean_count['rating_counts']>100)]
ratings_mean_count

,rating,rating_counts
title,,
2001: A Space Odyssey (1968),3.9,123
Airplane! (1980),3.8,106
Aladdin (1992),3.7,215
Alien (1979),4.0,127
Aliens (1986),3.9,125
...,...,...
While You Were Sleeping (1995),3.4,101
Who Framed Roger Rabbit? (1988),3.7,108
Willy Wonka & the Chocolate Factory (1971),3.8,148


In [395]:
ratings_mean_count = ratings_mean_count.sort_values(by = "rating" , ascending=False).head(10)
ratings_mean_count

,rating,rating_counts
title,,
"Godfather, The (1972)",4.5,200
"Shawshank Redemption, The (1994)",4.5,311
"Usual Suspects, The (1995)",4.4,201
"Godfather: Part II, The (1974)",4.4,135
Pulp Fiction (1994),4.3,324
Fargo (1996),4.3,224
One Flew Over the Cuckoo's Nest (1975),4.3,144
Schindler's List (1993),4.3,244
Taxi Driver (1976),4.2,118


#### calcul de la similarité cosinus en python

In [396]:

def square_rooted(x):
    return round(sqrt(sum([a*a for a in x])), 3)
def cosine_similarity(x,y):
    numerator =  sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return round(numerator/float(denominator), 3)

In [397]:
print(cosine_similarity([3,45,7,2],[2,54,13,15]))

0.972


In [398]:
#Autre methode pour calculer

# Vecteurs a et b
a = np.array([3, 45, 7, 2])
b = np.array([2, 54, 13, 15])

# Calcul de la similarité cosinus
dot_product = np.dot(a, b)
norm_a = np.linalg.norm(a)
norm_b = np.linalg.norm(b)
cosine_similarity =round(( dot_product / (norm_a * norm_b)),3)

cosine_similarity


0.972

In [399]:

# Deux phrases à comparer
sentence1 = "La maison est jolie et grande."
sentence2 = "La grande maison est très jolie."

# Prétraitement et conversion en vecteurs TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([sentence1, sentence2])

# Calcul de la similarité cosinus
cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

cosine_sim[0][0]


0.7168117414430619

In [400]:


pd.set_option('display.max_columns',100)
df= pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')
#df.head()

In [401]:
df.shape

(250, 38)

#### On va se servir du titre , du genre , du directeur , des acteurs et du resumé pour sortir des recommandations

In [402]:
df = df [['Title', 'Genre' , 'Director', 'Actors' , 'Plot']]

In [403]:
df['Actors'] = df['Actors'].map(lambda x: [actor.strip().lower().replace(' ', '') for actor in x.split(',')[:3]])
df['Genre'] = df['Genre'].map(lambda x: [genre.strip().lower() for genre in x.split(',')])
df['Director'] = df['Director'].map(lambda x: ''.join(x.split()).lower())

# Appliquer les transformations sur chaque ligne
for index, row in df.iterrows():
    df.at[index, 'Actors'] = [actor.strip().lower().replace(' ', '') for actor in row['Actors']]
    df.at[index, 'Director'] = ''.join(row['Director']).lower()

print(df[['Title', 'Actors', 'Director']].head())

                      Title                                      Actors  \
0  The Shawshank Redemption      [timrobbins, morganfreeman, bobgunton]   
1             The Godfather         [marlonbrando, alpacino, jamescaan]   
2    The Godfather: Part II       [alpacino, robertduvall, dianekeaton]   
3           The Dark Knight  [christianbale, heathledger, aaroneckhart]   
4              12 Angry Men      [martinbalsam, johnfiedler, leej.cobb]   

             Director  
0       frankdarabont  
1  francisfordcoppola  
2  francisfordcoppola  
3    christophernolan  
4         sidneylumet  


In [404]:

nltk.download('stopwords')
nltk.download('punkt')

df['Key_words']= ""
for index, row in df.iterrows():
    plot = row['Plot']
    r = Rake()
    r.extract_keywords_from_text(plot)
    key_words_dict_scores = r.get_word_degrees()
    df.at[index, 'Key_words'] = list(key_words_dict_scores.keys())

# Supprimer la colonne 'Plot' car elle n'est plus nécessaire
df.drop(columns=['Plot'], inplace=True)



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gaeta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gaeta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [405]:
key_words_dict_scores

defaultdict(<function rake_nltk.rake.Rake._build_word_co_occurance_graph.<locals>.<lambda>()>,
            {'mumbai': 3,
             'teen': 3,
             'reflects': 3,
             'upbringing': 1,
             'slums': 1,
             'accused': 1,
             'cheating': 1,
             'indian': 2,
             'version': 2,
             'wants': 1,
             'millionaire': 2,
             '?"': 2})

In [406]:
df['Key_words']

0      [two, imprisoned, men, bond, number, years, fi...
1      [aging, patriarch, organized, crime, dynasty, ...
2      [early, life, career, vito, corleone, 1920s, n...
3      [menace, known, joker, emerges, mysterious, pa...
4      [jury, holdout, attempts, prevent, miscarriage...
                             ...                        
245    [desperate, life, chronic, alcoholic, followed...
246    [20, something, supervising, staff, member, re...
247    [newspaper, editor, uses, every, trick, book, ...
248    [old, man, makes, long, journey, lawn, mover, ...
249    [mumbai, teen, reflects, upbringing, slums, ac...
Name: Key_words, Length: 250, dtype: object

In [407]:
df.set_index('Title', inplace =  True)
df.head()

,Genre,Director,Actors,Key_words
Title,,,,
The Shawshank Redemption,"[crime, drama]",frankdarabont,"[timrobbins, morganfreeman, bobgunton]","[two, imprisoned, men, bond, number, years, fi..."
The Godfather,"[crime, drama]",francisfordcoppola,"[marlonbrando, alpacino, jamescaan]","[aging, patriarch, organized, crime, dynasty, ..."
The Godfather: Part II,"[crime, drama]",francisfordcoppola,"[alpacino, robertduvall, dianekeaton]","[early, life, career, vito, corleone, 1920s, n..."
The Dark Knight,"[action, crime, drama]",christophernolan,"[christianbale, heathledger, aaroneckhart]","[menace, known, joker, emerges, mysterious, pa..."
12 Angry Men,"[crime, drama]",sidneylumet,"[martinbalsam, johnfiedler, leej.cobb]","[jury, holdout, attempts, prevent, miscarriage..."


In [408]:
# Transformer la colonne 'genres' en liste de genres
df['Genre'] = df['Genre'].apply(lambda x: x.split('|') if isinstance(x, str) else [])


In [409]:
# Créer une colonne bag_of_words vide
df['bag_of_words'] = ''

# Pondérer certaines colonnes pour donner plus d'importance aux caractéristiques
for index, row in df.iterrows():
    words = ''
    
    # Ajouter les genres avec un poids de 2
    words += (' '.join(row['Genre']) + ' ') * 2  # Répète les genres deux fois avec un espace entre chaque
    
    # Ajouter le directeur avec un poids de 1
    words += row['Director'] + ' '
    
    # Ajouter les acteurs avec un poids de 1.5 (répète chaque acteur 1 fois et demi)
    words += (' '.join(row['Actors']) + ' ') * int(1.5)  # On ajoute 1.5 fois (en pratique, cela revient à 1)
    
    # Ajouter les mots-clés avec un poids de 1
    words += ' '.join(row['Key_words']) + ' '
    
    # Mettre à jour la colonne 'bag_of_words' pour la ligne actuelle
    df.at[index, 'bag_of_words'] = words.strip()

# Supprimer les colonnes qui ne sont pas bag_of_words
df.drop(columns=[col for col in df.columns if col != 'bag_of_words'], inplace=True)

print(df.head())


                                                               bag_of_words
Title                                                                      
The Shawshank Redemption  frankdarabont timrobbins morganfreeman bobgunt...
The Godfather             francisfordcoppola marlonbrando alpacino james...
The Godfather: Part II    francisfordcoppola alpacino robertduvall diane...
The Dark Knight           christophernolan christianbale heathledger aar...
12 Angry Men              sidneylumet martinbalsam johnfiedler leej.cobb...


In [410]:
count = CountVectorizer()
count_matrix =  count.fit_transform(df['bag_of_words'])

In [411]:
count_matrix

<250x2952 sparse matrix of type '<class 'numpy.int64'>'
	with 4701 stored elements in Compressed Sparse Row format>

In [412]:
c = count_matrix.todense()

In [413]:
c

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [414]:
print(count_matrix[0,:])

  (0, 1006)	1
  (0, 2669)	1
  (0, 1805)	1
  (0, 304)	1
  (0, 2756)	1
  (0, 1264)	1
  (0, 1728)	1
  (0, 309)	1
  (0, 1893)	1
  (0, 2941)	1
  (0, 964)	1
  (0, 2474)	1
  (0, 885)	1
  (0, 2168)	1
  (0, 59)	1
  (0, 516)	1
  (0, 652)	1


In [415]:
cosine_sim = cosine_similarity(count_matrix , count_matrix)
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.22537447, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.22537447, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [416]:
indices = pd.Series(df.index)
indices[:20]

0                              The Shawshank Redemption
1                                         The Godfather
2                                The Godfather: Part II
3                                       The Dark Knight
4                                          12 Angry Men
5                                      Schindler's List
6         The Lord of the Rings: The Return of the King
7                                          Pulp Fiction
8                                            Fight Club
9     The Lord of the Rings: The Fellowship of the Ring
10                                         Forrest Gump
11       Star Wars: Episode V - The Empire Strikes Back
12                                            Inception
13                The Lord of the Rings: The Two Towers
14                      One Flew Over the Cuckoo's Nest
15                                           Goodfellas
16                                           The Matrix
17                   Star Wars: Episode IV - A N

In [417]:
def recommandation(title, cosine_sim=cosine_sim):
    # Liste pour stocker les films recommandés
    recommended_movies = []

    # Trouver l'index du film donné dans le DataFrame
    idx = indices[indices == title].index[0]

    # Créer une série de scores de similarité cosinus pour le film donné
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending=False)

    # Récupérer les index des 10 films les plus similaires (hors le film lui-même)
    top_10_indexes = list(score_series.iloc[1:11].index)

    # Afficher les index des films recommandés (à des fins de débogage)
    print(top_10_indexes)

    # Ajouter les titres des films recommandés à la liste recommended_movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])

    # Retourner la liste des films recommandés
    return recommended_movies

In [418]:
recommendations = recommandation("The Dark Knight")
print("Recommendations for 'The Dark Knight':")
print(recommendations)


[51, 89, 40, 203, 36, 219, 29, 215, 9, 187]
Recommendations for 'The Dark Knight':
['The Dark Knight Rises', 'Batman Begins', 'The Prestige', 'Guardians of the Galaxy Vol. 2', 'Terminator 2: Judgment Day', 'The Avengers', 'The Green Mile', 'Sin City', 'The Lord of the Rings: The Fellowship of the Ring', 'Out of the Past']
